In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]='0'
os.environ["HF_HOME"]="~/codes/.cache/huggingface"

In [2]:
from accelerate.utils import ProjectConfiguration, set_seed

In [3]:
import torch
import random
import numpy as np

def set_seeds(seed):
    set_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_seeds(42)

In [4]:
from torchvision import transforms
from diffusers import DDPMPipeline, DDIMPipeline, DDPMScheduler, DDIMScheduler, UNet2DModel

In [5]:
import pickle

In [6]:
class Args():
    
    model_config_name_or_path="config.json"
    
    dataset_name="cifar10"

    resolution=32
    center_crop=True
    random_flip=False

    dataloader_num_workers=8
    
    seed=42
    gen_seed=0    

    train_batch_size=256

    ddpm_num_steps=1000
    ddpm_num_inference_steps=50
    ddpm_beta_schedule='linear'

args=Args()
args.model_config_name_or_path

'config.json'

In [7]:
with open('./data/indices/5000-0.5/counter/idx-gen-sampled.pkl', 'rb') as handle:
    test_index = pickle.load(handle)
test_index

[521,
 737,
 740,
 660,
 411,
 678,
 626,
 513,
 859,
 136,
 811,
 76,
 636,
 973,
 938,
 899,
 280,
 883,
 761,
 319,
 549,
 174,
 371,
 527,
 210,
 235,
 101,
 986,
 902,
 947,
 346,
 139,
 621,
 499,
 370,
 198,
 687,
 584,
 901,
 59,
 328,
 96,
 312,
 974,
 299,
 277,
 924,
 601,
 439,
 837,
 570,
 879,
 261,
 578,
 23,
 30,
 617,
 10,
 221,
 820]

In [8]:
index_image_list = []
for index, my_i in enumerate(test_index):
        
    print(index, my_i)
    k_image_list = []
    for k in [
        1000
    ]:
        method_image_list = []
        for method in ['Random', 'TRAK', 'Ours']:
            # If passed along, set the training seed now.
            if args.seed is not None:
                set_seeds(args.seed)
            ####    
            print(args.model_config_name_or_path)
            config = UNet2DModel.load_config(args.model_config_name_or_path)
            config['resnet_time_scale_shift'] = 'scale_shift'
                
            model = UNet2DModel.from_config(config)
            print(model.dtype)
            ####
            noise_scheduler = DDPMScheduler(num_train_timesteps=args.ddpm_num_steps, beta_schedule=args.ddpm_beta_schedule)
            ddim_noise_scheduler = DDIMScheduler.from_config(noise_scheduler.config)
            ####    
            args.model_path = "./saved/5000-0.5/counter/ddpm-sub-{}-{}-{}-42".format(index, k, method)
            model_path = '{}/unet/diffusion_pytorch_model.bin'.format(args.model_path)
            print(model_path)
            model.load_state_dict(torch.load(model_path))
            model.cuda()
            model.eval()
    ####
            set_seeds(42)
            pipeline = DDIMPipeline(
                unet=model,
                scheduler=noise_scheduler,
            ).to('cuda')
            print(pipeline.device)
            print('DDIM!')
            
            generator = [torch.Generator('cpu').manual_seed(my_i)]
                    
            images = pipeline(
                        generator=generator,
                        batch_size=1,
                        num_inference_steps=50,
                        output_type="numpy",
                        eta=0.0,
            ).images
            images = pipeline.numpy_to_pil(images)
        
            method_image_list.append(images[0])
        k_image_list.append(method_image_list)
    index_image_list.append(k_image_list)

0 521
config.json
torch.float32
./saved/5000-0.5/counter/ddpm-sub-0-1000-Random-42/unet/diffusion_pytorch_model.bin
cuda:0
DDIM!


  0%|          | 0/50 [00:00<?, ?it/s]

config.json
torch.float32
./saved/5000-0.5/counter/ddpm-sub-0-1000-TRAK-42/unet/diffusion_pytorch_model.bin
cuda:0
DDIM!


  0%|          | 0/50 [00:00<?, ?it/s]

config.json
torch.float32
./saved/5000-0.5/counter/ddpm-sub-0-1000-Ours-42/unet/diffusion_pytorch_model.bin
cuda:0
DDIM!


...

  0%|          | 0/50 [00:00<?, ?it/s]

config.json
torch.float32
./saved/5000-0.5/counter/ddpm-sub-59-1000-TRAK-42/unet/diffusion_pytorch_model.bin
cuda:0
DDIM!


  0%|          | 0/50 [00:00<?, ?it/s]

config.json
torch.float32
./saved/5000-0.5/counter/ddpm-sub-59-1000-Ours-42/unet/diffusion_pytorch_model.bin
cuda:0
DDIM!


  0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
with open("./counter/saved.pkl", 'wb') as handle:
    pickle.dump(index_image_list, handle)